네이버영화평점
==
- 감성분석
- 네이버 영화평점 (Naver sentiment movie corpus v.1.0) 데이터(https://github.com/e9t/nsmc)
- 영화 리뷰 20만건이 저장됨. 각 평가 데이터는 0(부정), 1(긍정)으로 label 됨.

### 한글 자연어 처리
- KoNLPy(“코엔엘파이”라고 읽습니다)는 한국어 정보처리를 위한 파이썬 패키지입니다.
- konlpy 패키지에서 제공하는 Twitter라는 문서 분석 라이브러리 사용 (트위터 분석 뿐 아니라 한글 텍스트 
  처리도 가능)
- colab 사용 권장

# 로지스틱회귀를 이용한 감성분석

In [1]:
!pip install konlpy

In [4]:
# Curl:
# curl is a tool to transfer data from or to a server, using one of the supported protocols (HTTP, HTTPS, FTP,
# FTPS, SCP, SFTP, TFTP, DICT, TELNET, LDAP or FILE). The command is designed to work without user interaction.
# 
# curl -L : (HTTP/HTTPS) If the server reports that the requested page has moved to a different location 
# (indicated with a Location: header and a 3XX response code), this option will make curl redo the request 
# on the new place.

In [5]:
# 네이버 영화 평점 데이터 다운로드
!curl -L https://bit.ly/2X9Owwr -o ratings_train.txt
!curl -L https://bit.ly/2WuLd5I -o ratings_test.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   152  100   152    0     0    136      0  0:00:01  0:00:01 --:--:--   138
100   152  100   152    0     0    136      0  0:00:01  0:00:01 --:--:--   138

  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0

  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0

  0     0    0     0    0     0      0      0 --:--:--  0:00:02 --:--:--     0
  8 14.0M    8 1262k    0     0   406k      0  0:00:35  0:00:03  0:00:32 1922k
 25 14.0M   25 3710k    0     0   902k      0  0:00:15  0:00:04  0:00:11 2239k
 46 14.0M   46 6734k    0     0  1317k      0  0:00:10  0:00:05  0:00:05 2534k
 66 14.0M   66 9662k    0     0  1581k      0  0

In [6]:
# KoNLPy(“코엔엘파이”라고 읽습니다)는 한국어 정보처리를 위한 파이썬 패키지입니다.

In [9]:
!dir

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: 264E-2F1C

 C:\Users\rta_note\Desktop\Lab_M34 디렉터리

2019-09-06  오후 03:13    <DIR>          .
2019-09-06  오후 03:13    <DIR>          ..
2019-08-31  오후 02:10    <DIR>          .ipynb_checkpoints
2019-07-20  오후 01:21             3,258 baseline-script.csv
2019-07-21  오후 07:28           659,240 bike_train.csv
2019-07-20  오후 01:17             5,221 decision-tree.dot
2019-08-05  오후 06:13             8,587 digit2.png
2019-08-05  오후 06:20             9,344 digit2_2.png
2019-08-05  오후 06:13             7,431 digit9.png
2019-08-05  오후 06:14             8,408 digit9_2.png
2019-08-05  오후 06:21            33,284 digit9_3.png
2019-08-05  오후 06:21            24,365 digit9_4.png
2019-08-05  오후 07:09             6,067 digits.pkl
2019-08-05  오후 05:02         5,955,180 face-detect.png
2019-08-05  오후 05:03            58,609 face-pickup.png
2019-08-05  오후 05:00         1,774,696 families.jpg
2019-07-20  오전 10:33             8,917 gg_34_kNN.ipynb
2019-07-20  오후 01:01       

In [10]:
import konlpy
import pandas as pd
from konlpy.tag import Twitter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
# from sklearn.pipeline import make_pipeline
# import pickle
# import os.path

# 데이터 로드
# keep_default_na: Whether or not to include the default NaN values when parsing the data
# -> False: no strings will be parsed as NaN.

df_train = pd.read_csv('ratings_train.txt', delimiter='\t', keep_default_na=False)
df_test = pd.read_csv('ratings_test.txt', delimiter='\t', keep_default_na=False)

df_train.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [11]:
# text_train, y_train = df_train['document'].as_matrix(), df_train['label'].as_matrix()
# text_test, y_test = df_test['document'].as_matrix(), df_test['label'].as_matrix()

In [12]:
text_train, y_train = df_train['document'].values, df_train['label'].values
text_test, y_test = df_test['document'].values, df_test['label'].values

In [13]:
def twitter_tokenizer(text):
    return twitter_tag.morphs(text)

In [14]:
twitter_tag = Twitter()

cv = TfidfVectorizer(tokenizer=twitter_tokenizer, min_df=5)

C:\Users\rta_note\Anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


JVMNotFoundException: No JVM shared library file (jvm.dll) found. Try setting up the JAVA_HOME environment variable properly.

In [7]:
lr = LogisticRegression()

x_train = cv.fit_transform(text_train)
x_test = cv.transform(text_test)
result = lr.fit(x_train,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [9]:
len(cv.vocabulary_)

22414

In [15]:
feature_names = cv.get_feature_names()
print(feature_names[3000:3010])

['기억나는게', '기억나는데', '기억나서', '기억나요', '기억나지', '기억난다', '기억남', '기억남는', '기억남는건', '기억상실증']


In [19]:
print("훈련 데이터 점수 : ", result.score(x_train, y_train))
print("테스트 데이터 점수 : ", result.score(x_test, y_test))

훈련 데이터 점수 :  0.88164
테스트 데이터 점수 :  0.85088


# LSTM을 이용한 분석

In [0]:
text_data, y_data = df_data['document'].as_matrix(), df_data['label'].as_matrix()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [0]:
from konlpy.tag import Twitter
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from sklearn import model_selection, metrics
import numpy as np
import pickle
import os.path
import keras.backend.tensorflow_backend as K

# 토큰 파서
def twitter_tokenizer(text):
    return twitter_tag.morphs(text)

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [0]:
twitter_tag = Twitter()

In [0]:
cv = TfidfVectorizer(tokenizer=twitter_tokenizer)

In [0]:
!ls X*

X_data.pickle	 X_test_2.pickle  X_train_1.pickle
X_test_1.pickle  X_test.pickle	  X_train_2.pickle


In [0]:
# Tfidf 생성과 저장 - 단 파일이 없을 때만 !!
if not os.path.isfile("X_data.pickle"): 
    print('file does not exists')
    X_data = cv.fit_transform(text_data)
    pickle.dump(X_train, open("X_data.pickle", "wb"))

In [0]:
# 저장된 tfidf vector 데이터 읽기
with open('X_data.pickle', 'rb') as f:
    X_data = pickle.load(f)

In [0]:
!ls -al X*

-rw-rw-r-- 1 hjkim hjkim 24433014 Oct 28 18:39 X_data.pickle
-rw-rw-r-- 1 hjkim hjkim  7782344 Oct 28 19:03 X_test_1.pickle
-rw-rw-r-- 1 hjkim hjkim  5610600 Oct 28 18:56 X_test_2.pickle
-rw-rw-r-- 1 hjkim hjkim  8170628 Oct 28 18:40 X_test.pickle
-rw-rw-r-- 1 hjkim hjkim 23745856 Oct 28 19:02 X_train_1.pickle
-rw-rw-r-- 1 hjkim hjkim 14084384 Oct 28 18:55 X_train_2.pickle


In [0]:
# one-hot encoding
Y_data = np_utils.to_categorical(y_data, 2)

In [0]:
X_train = X_data[:100000]
X_test = X_data[100000:]

Y_train = Y_data[:100000]
Y_test = Y_data[100000:]

In [0]:
max_words = 61070 
nb_classes = 2
batch_size = 1024
nb_epoch = 5

In [0]:
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
Y_train.shape[1]

(100000, 104160)
(50000, 104160)
(100000, 2)


2

In [0]:
# LSTM 학습을 위한 데이터 재배열 (Time step)
X_train_rnn = X_train.A.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test_rnn = X_test.A.reshape((X_test.shape[0], 1, X_test.shape[1]))

print(X_train_rnn.shape)
print(X_test_rnn.shape)

(100000, 1, 104160)
(50000, 1, 104160)


In [0]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import np_utils

def build_LSTM_model():
    model = Sequential()
    model.add(LSTM(128, input_shape=(X_train_rnn.shape[1], X_train_rnn.shape[2]), return_sequences=True))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(LSTM(128))
    model.add(Dropout(0.2))
    model.add(Dense(Y_train.shape[1], activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [0]:
import keras.backend.tensorflow_backend as K
with K.tf.device('/GPU:1'):
    model_lstm = KerasClassifier(
    build_fn=build_LSTM_model, 
    epochs=nb_epoch, 
    batch_size=batch_size)
    
    model_lstm.fit(X_train_rnn, Y_train)

Epoch 1/5
100000/100000 [==============================] - 252s 3ms/step - loss: 0.5463 - acc: 0.7826
Epoch 2/5
100000/100000 [==============================] - 238s 2ms/step - loss: 0.2854 - acc: 0.8830
Epoch 3/5
100000/100000 [==============================] - 239s 2ms/step - loss: 0.2076 - acc: 0.9218
Epoch 4/5
100000/100000 [==============================] - 238s 2ms/step - loss: 0.1604 - acc: 0.9420
Epoch 5/5
100000/100000 [==============================] - 239s 2ms/step - loss: 0.1293 - acc: 0.9543


In [0]:
y = model_lstm.predict(X_train_rnn)
y_train = y_data[:100000]
ac_score = metrics.accuracy_score(y_train, y)
print("훈련 셋 정답률 =", ac_score)

훈련 셋 정답률 = 0.96941


In [0]:
# predict 함수는 예측 결과를 0 or 1로 출력하므로
# 학습과정에서 사용한 Y_train, Y_test 변수로 정확도 측정이 안됨
# Y_train, Y_test는 [0, 1], [1, 0]의 형태로 해당하는 감정 컬럼(class)은 1, 다른 컬럼은 0으로 표시됨
# 초기 y_data에 저장된 값을 그대로 활용하여 정확도를 측정

print("y : ", y)
print("Y_train[0] : ", Y_train)
print("y_train[0] : ", y_data)

y :  [0 1 0 ... 0 1 1]
Y_train[0] :  [[1. 0.]
 [0. 1.]
 [1. 0.]
 ...
 [1. 0.]
 [0. 1.]
 [0. 1.]]
y_train[0] :  [0 1 0 ... 0 1 0]


In [0]:
y = model_lstm.predict(X_test_rnn)
y_test = y_data[100000:]
ac_score = metrics.accuracy_score(y_test, y)
print("테스트 셋 정답률 =", ac_score)

테스트 셋 정답률 = 0.83784


## 연습

In [9]:
import numpy as np
import pandas as pd
df = pd.DataFrame(np.arange(12).reshape(3,4)); df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11


In [12]:
df.as_matrix()   # Convert the frame to its Numpy-array representation.

C:\Users\rta_note\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [16]:
df.values       # using this is recommended

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [19]:
df[0].as_matrix(), df[0].values

C:\Users\rta_note\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


(array([0, 4, 8]), array([0, 4, 8]))